In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
import numpy as np

# josh statquest vid
video_id = "FgakZw6K1QQ"
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages = ['en'])


In [2]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)

print(type(transcript))
pp.pprint(transcript[0])

<class 'list'>
{   'duration': 6.5,
    'start': 0.5,
    'text': 'StatQuest breaks it down into bite-sized pieces, hooray!'}


In [3]:
print(len(transcript))
pp.pprint(transcript[-1])

180
{   'duration': 5.475,
    'start': 1307.787,
    'text': 'link to my bandcamp page is in the lower right corner and in the '
            'description below. All right until next time quest on!'}


### The transcript has 180 chunks in the way it is formatted. 
Are these chunks proper sentences or paragraphs? What is the length of these texts? Are they appropriate to directly plug into the vector index? Would they help answer the user query or do they need more processing?

In [20]:
import pandas as pd

trans_df = pd.DataFrame(transcript)

In [22]:
trans_df['text_len'] = trans_df.text.str.len()

In [23]:
trans_df.head()

,text,start,duration,text_len
0,StatQuest breaks it down into bite-sized piece...,0.50,6.500,56
1,"Hello, I'm Josh Starmer and welcome to StatQue...",8.00,6.040,82
2,go through Principal Component Analysis (PCA) ...,14.09,6.389,106
3,"You'll learn about what PCA does, how it does ...",23.00,5.064,105
4,Let's start with a simple data set.,30.70,2.100,35


In [24]:
trans_df.describe()

,start,duration,text_len
count,180.000000,180.000000,180.000000
mean,638.178572,6.322017,77.183333
std,378.839422,1.914191,19.808997
min,0.500000,1.900000,24.000000
25%,315.050500,5.085750,67.500000
50%,631.800000,6.388000,82.000000
75%,957.815750,7.619750,85.000000
max,1307.787000,11.063000,120.000000


The max length of the chunk text is 120 which is indeed a small number for most of the embedding models which come with there own input text length limitations.
Infact the size is appropriate enough for us to later explore merging the relevant chunks on the basis of neighboring chunk similarity in case we need to improve the quality of RAG based answer generation.
In other approach, we may also explore merging all the chunks into a single long text which we can choose to chunk in a custom manner.

For now we will start with the current chunks.

We found that these chunks are too small and broken up at completely random places which makes it hard to retrieve relevant chunks and for the LLM to make any sense. Now the experiment I am trying is to merge 4 chunks to form 1.

In [22]:
video_id = "FgakZw6K1QQ"
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages = ['en'])

new_chunks = []
for i in range(0, len(transcript), 4):
    new_chunks.append({"text":" ".join([chunk['text'] for chunk in transcript[i:i+4]])})


In [23]:
len(transcript)

180

In [24]:
len(new_chunks)

45

In [26]:
new_chunks[:5]

[{'text': "StatQuest breaks it down into bite-sized pieces, hooray! Hello, I'm Josh Starmer and welcome to StatQuest. In this StatQuest we're going to go through Principal Component Analysis (PCA) one step at a time using Singular Value Decomposition (SVD). You'll learn about what PCA does, how it does it, and how to use it to get deeper insight into your data."},
 {'text': "Let's start with a simple data set. We've measured the transcription of two genes, Gene 1 and Gene 2,\xa0in\xa06\xa0different mice. Note: If you're not into mice and genes, think of the mice as individual samples and the genes as variables that we measure for each sample."},
 {'text': "For example, the samples could be students in high school and the variables could be test scores in math and reading, or the samples could be businesses and the variables could be market capitalization and the number of employees. Okay, now we're back to mice and genes, because I'm a geneticist and I work in a genetics department."},